In [ ]:
# import pandas as pd

# df = pd.read_parquet("expanded_df.parquet")
# df = df[
#     (df["当前法律状态"] == "专利申请权、专利权的转移")
#     | (df["当前法律状态"] == "专利申请权、专利权的转移(专利权的转移);")
#     | (df["当前法律状态"] == "专利申请权、专利权的转移(专利申请权的转移);")
# ]
# df.to_parquet("专利申请权、专利权的转移2.parquet")
# df['当前法律状态'].value_counts().to_csv('法律状态.csv', encoding='utf-8-sig')
# df = df[df['当前法律状态'] == '专利申请权、专利权的转移']
# df.to_parquet("专利申请权、专利权的转移.parquet")


In [ ]:
# 提取登记生效日并删除相关信息
import re

# 提取登记生效日
def extract_registration_date(x):
    if pd.isna(x):
        return None
    x_str = str(x)
    # 匹配 YYYYMMDD 格式
    match1 = re.search(r'登记生效日:(\d{8})', x_str)
    if match1:
        return match1.group(1)
    # 匹配 YYYY.MM.DD 格式
    match2 = re.search(r'登记生效日:(\d{4})\.(\d{2})\.(\d{2})', x_str)
    if match2:
        return match2.group(1) + match2.group(2) + match2.group(3)
    # 匹配 YYYY.M.D 格式（单位数的月份和日期）
    match3 = re.search(r'登记生效日:(\d{4})\.(\d{1,2})\.(\d{1,2})', x_str)
    if match3:
        year = match3.group(1)
        month = match3.group(2).zfill(2)  # 补零到两位数
        day = match3.group(3).zfill(2)    # 补零到两位数
        return year + month + day
    return None

df['登记生效日'] = df['状态描述信息'].apply(extract_registration_date)

# 删除状态描述信息中的登记生效日信息
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'登记生效日:(?:\d{8}|\d{4}\.\d{1,2}\.\d{1,2})\s*;?\s*', '', str(x)) if pd.notna(x) else x
)
df = df[df['登记生效日']>='20100101']
df.to_parquet('申请在10年之前的记录')

In [1]:
import pandas as pd

df = pd.read_parquet("专利申请权、专利权的转移.parquet")
df2 = pd.read_parquet("申请在10年之前的记录.parquet")
# 添加新列判断是专利申请权的转移还是专利权的转移
df["专利权"] = df["状态描述信息"].apply(
    lambda x: 1 if str(x).startswith("专利权的转移") else 0
)
df2["专利权"] = df2["状态描述信息"].apply(
    lambda x: 1
    if str(x).startswith("专利权的转移")
    | str(x).startswith("专利申请权、专利权的转移(专利权的转移);")
    else 0
)


In [2]:
# 删除状态描述信息中"两位数字;"或"两位数字 ;"及之前的内容
import re
df['状态描述信息'] = df['状态描述信息'].apply(lambda x: re.sub(r'^.*?\d{2}\s*;', '', str(x)) if pd.notna(x) else x)
# df2['状态描述信息'] = df2['状态描述信息'].apply(lambda x: re.sub(r'^.*?\d{2}(?!\d)\s*;', '', str(x)) if pd.notna(x) else x)
# 删除df2状态描述信息中的专利申请权、专利权的转移相关前缀
df2['状态描述信息'] = df2['状态描述信息'].apply(
    lambda x: re.sub(r'^专利申请权、专利权的转移\(专利权的转移\);|^专利申请权、专利权的转移\(专利申请权的转移\);', '', str(x)) if pd.notna(x) else x
)




In [3]:
# 提取登记生效日并删除相关信息
import re

# 提取登记生效日
df['登记生效日'] = df['状态描述信息'].apply(
    lambda x: re.search(r'登记生效日:(\d{8})', str(x)).group(1) if re.search(r'登记生效日:(\d{8})', str(x)) else None
)

# 删除状态描述信息中的登记生效日信息
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'登记生效日:\d{8}\s*;?\s*', '', str(x)) if pd.notna(x) else x
)


In [4]:
df['2010年之前申请'] = 0
df2['2010年之前申请'] = 1
df = pd.concat([df, df2])

In [5]:
# 统一变更前和变更后的格式
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'变更前(?!\s*;)', '变更前 ;', str(x)) if pd.notna(x) else x
)
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'变更后(?!\s*;)', '变更后 ;', str(x)) if pd.notna(x) else x
)


In [6]:
# 删除状态描述信息中的空格，并将分号替换为换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: str(x).replace(' ', '').replace(';', '\n') if pd.notna(x) else x
)



In [7]:
# 确保"变更后"前面有换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'(?<!\n)变更后', '\n变更后', str(x)) if pd.notna(x) else x
)
# 确保除了最开头的"变更前"外，其他"变更前"前面有换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'(?<!^)(?<!\n)变更前', '\n变更前', str(x)) if pd.notna(x) else x
)


In [8]:
# 删除状态描述信息中"变更前\n国家或地区"及之后的所有内容
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'\n变更前\n国家或地区.*', '', str(x), flags=re.DOTALL) if pd.notna(x) else x
)

# 删除状态描述信息中"变更后\n国家或地区"及之后的所有内容
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: re.sub(r'\n变更后\n国家或地区.*', '', str(x), flags=re.DOTALL) if pd.notna(x) else x
)

# 删除状态描述信息末尾的换行符
df['状态描述信息'] = df['状态描述信息'].apply(
    lambda x: str(x).rstrip('\n') if pd.notna(x) else x
)




In [9]:
# 从2010年之前申请的数据中，去掉状态描述信息内部包含"邮编"的行
def remove_zipcode_lines(status_info):
    if pd.isna(status_info):
        return status_info
    
    # 按换行符分割，过滤掉包含"邮编"的行，然后重新组合
    lines = str(status_info).split('\n')
    filtered_lines = [line for line in lines if '邮编' not in line and '邮政编码' not in line]
    return '\n'.join(filtered_lines)

# 只对2010年之前申请的数据进行处理
mask_2010_before = df['2010年之前申请'] == 1
df.loc[mask_2010_before, '状态描述信息'] = df.loc[mask_2010_before, '状态描述信息'].apply(remove_zipcode_lines)



In [10]:
# 根据状态描述信息的特征设置type1字段
def set_type1(status_info):
    if pd.isna(status_info):
        return 0
    
    # 转换为字符串并按换行符分割
    lines = str(status_info).split('\n')
    
    # 检查是否以"变更前"开头且行数为6
    if len(lines) == 6 and lines[0].startswith('变更前'):
        return 1
    else:
        return 0

df['type1'] = df['状态描述信息'].apply(set_type1)

# 根据状态描述信息的特征设置type2字段
def set_type2(status_info):
    if pd.isna(status_info):
        return 0
    
    # 转换为字符串并按换行符分割
    lines = str(status_info).split('\n')
    
    # 检查是否以"变更前"开头，第二行以"专利权人"或"申请人"开头，最后一行以"地址"开头，且行数为8
    if (len(lines) == 8 and 
        lines[0] == '变更前' and 
        lines[2] == '变更前' and
        lines[4] == '变更后' and
        lines[6] == '变更后'):
        return 1
    else:
        return 0

df['type2'] = df['状态描述信息'].apply(set_type2)
# 根据状态描述信息的特征设置type3字段
def set_type3(status_info):
    if pd.isna(status_info):
        return 0
    
    # 转换为字符串并按换行符分割
    lines = str(status_info).split('\n')
    
    # 检查是否行数为10，第一行为"变更事项:专利权人"或"变更事项:申请人"，第6行为"变更事项:地址"
    if (len(lines) == 10 and 
        (lines[0] == '变更事项:专利权人' or lines[0] == '变更事项:申请人') and 
        lines[5] == '变更事项:地址'):
        return 1
    else:
        return 0

df['type3'] = df['状态描述信息'].apply(set_type3)


df['type4'] = 1-df['type1']-df['type2']-df['type3']

# 根据状态描述信息的特征设置type5字段
def set_type5(row):
    if row['type4'] == 1 and pd.notna(row['状态描述信息']):
        # 检查状态描述信息是否以"变更事项"开头
        if str(row['状态描述信息']).startswith('变更事项'):
            return 1
    return 0

df['type5'] = df.apply(set_type5, axis=1)

# 更新type4，使其不包含type5的情况
df['type4'] = df['type4'] - df['type5']


In [11]:
type1 = df[df['type1'] == 1]
# 为type1数据框新增四列并提取相应信息
def extract_info(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 确保有足够的行数
    if len(lines) >= 6:
        # 提取第二行":"后面的内容作为前权利人
        if ':' in lines[1]:
            前权利人 = lines[1].split(':', 1)[1].strip()
        
        # 提取第三行":"后面的内容作为前地址
        if ':' in lines[2]:
            前地址 = lines[2].split(':', 1)[1].strip()
        
        # 提取第五行":"后面的内容作为后权利人
        if ':' in lines[4]:
            后权利人 = lines[4].split(':', 1)[1].strip()
        
        # 提取第六行":"后面的内容作为后地址
        if ':' in lines[5]:
            后地址 = lines[5].split(':', 1)[1].strip()
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type1[['前权利人', '前地址', '后权利人', '后地址']] = type1['状态描述信息'].apply(
    lambda x: pd.Series(extract_info(x))
)


C:\Users\zy222\AppData\Local\Temp\ipykernel_30932\1859307967.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type1[['前权利人', '前地址', '后权利人', '后地址']] = type1['状态描述信息'].apply(
C:\Users\zy222\AppData\Local\Temp\ipykernel_30932\1859307967.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type1[['前权利人', '前地址', '后权利人', '后地址']] = type1['状态描述信息'].apply(
C:\Users\zy222\AppData\Local\Temp\ipykernel_30932\1859307967.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [12]:
type2 = df[df['type2'] == 1]
# 为type2数据框新增四列并提取相应信息
def extract_info_type2(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 确保有足够的行数
    if len(lines) >= 8:
        # 提取第二行":"后面的内容作为前权利人
        if ':' in lines[1]:
            前权利人 = lines[1].split(':', 1)[1].strip()
        
        # 提取第四行":"后面的内容作为前地址
        if ':' in lines[3]:
            前地址 = lines[3].split(':', 1)[1].strip()
        
        # 提取第六行":"后面的内容作为后权利人
        if ':' in lines[5]:
            后权利人 = lines[5].split(':', 1)[1].strip()
        
        # 提取第八行":"后面的内容作为后地址
        if ':' in lines[7]:
            后地址 = lines[7].split(':', 1)[1].strip()
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type2 = type2.copy()  # 避免SettingWithCopyWarning
type2[['前权利人', '前地址', '后权利人', '后地址']] = type2['状态描述信息'].apply(
    lambda x: pd.Series(extract_info_type2(x))
)


In [13]:
type3 = df[df['type3'] == 1]
# 为type3数据框新增四列并提取相应信息
def extract_info_type3(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 确保有足够的行数
    if len(lines) >= 10:
        # 提取第三行":"后面的内容作为前权利人
        if len(lines) > 2 and ':' in lines[2]:
            前权利人 = lines[2].split(':', 1)[1].strip()
        
        # 提取第五行":"后面的内容作为后权利人
        if len(lines) > 4 and ':' in lines[4]:
            后权利人 = lines[4].split(':', 1)[1].strip()
        
        # 提取第八行":"后面的内容作为前地址
        if len(lines) > 7 and ':' in lines[7]:
            前地址 = lines[7].split(':', 1)[1].strip()
        
        # 提取第十行":"后面的内容作为后地址
        if len(lines) > 9 and ':' in lines[9]:
            后地址 = lines[9].split(':', 1)[1].strip()
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type3 = type3.copy()  # 避免SettingWithCopyWarning
type3[['前权利人', '前地址', '后权利人', '后地址']] = type3['状态描述信息'].apply(
    lambda x: pd.Series(extract_info_type3(x))
)


In [14]:
type4 = df[df['type4'] == 1]
# 为type4数据框新增四列并提取相应信息
def extract_info_type4(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 查找包含"地址:"的行
    address_lines = []
    for line in lines:
        if '地址:' in line:
            address_content = line.split('地址:', 1)[1].strip()
            address_lines.append(address_content)
    
    # 根据找到的地址行数量分配前地址和后地址
    if len(address_lines) >= 1:
        前地址 = address_lines[0]
    if len(address_lines) >= 2:
        后地址 = address_lines[1]
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type4 = type4.copy()  # 避免SettingWithCopyWarning
type4[['前权利人', '前地址', '后权利人', '后地址']] = type4['状态描述信息'].apply(
    lambda x: pd.Series(extract_info_type4(x))
)


In [15]:
type5 = df[df['type5'] == 1]
# 为type5数据框新增四列并提取相应信息
def extract_info_type5(status_info):
    if pd.isna(status_info):
        return None, None, None, None
    
    lines = str(status_info).split('\n')
    
    # 初始化返回值
    前权利人 = None
    前地址 = None
    后权利人 = None
    后地址 = None
    
    # 查找包含"变更事项:地址"的行
    for i, line in enumerate(lines):
        if '变更事项:地址' in line:
            # 前地址是该行后第二行
            if i + 2 < len(lines):
                line_content = lines[i + 2].strip()
                if ':' in line_content:
                    前地址 = line_content.split(':', 1)[1].strip()
                else:
                    前地址 = line_content
            # 后地址是该行后第四行
            if i + 4 < len(lines):
                line_content = lines[i + 4].strip()
                if ':' in line_content:
                    后地址 = line_content.split(':', 1)[1].strip()
                else:
                    后地址 = line_content
            break
    
    return 前权利人, 前地址, 后权利人, 后地址

# 应用函数并创建新列
type5 = type5.copy()  # 避免SettingWithCopyWarning
type5[['前权利人', '前地址', '后权利人', '后地址']] = type5['状态描述信息'].apply(
    lambda x: pd.Series(extract_info_type5(x))
)


In [18]:
df_out = pd.concat([type1, type2, type3, type4, type5])
df_out = df_out[['当前法律状态','前权利人','前地址','后权利人','后地址','申请号','登记生效日']]
df_out.to_parquet("专利转让.parquet")

In [20]:
df_out.sample(20).to_csv("专利转让sample.csv",index=False)